# 1. Carga de datos desde Kedro

En este notebook ejecutamos clustering usando los datos limpios del pipeline de Kedro: `diabetes_cleaned`.

In [ ]:
# ==== Inicializar Kedro correctamente dentro del notebook ====

import pandas as pd
from pathlib import Path
from kedro.framework.session import KedroSession
from kedro.framework.startup import ProjectMetadata

# Detectar el directorio raíz del proyecto (2 niveles arriba del notebook)
project_path = Path.cwd().resolve().parents[1]

# Cargar metadata del proyecto desde pyproject.toml
metadata = ProjectMetadata.from_path(project_path)

# Crear sesión
session = KedroSession.create(metadata.package_name, project_path)

# Cargar contexto
context = session.load_context()

# Acceder al catálogo
catalog = context.catalog

print("Kedro cargado correctamente ✓")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:20                                                                                   │
│                                                                                                  │
│   17 context = session.load_context()                                                            │
│   18                                                                                             │
│   19 # Acceder al catálogo                                                                       │
│ ❱ 20 catalog = context.catalog                                                                   │
│   21                                                                                             │
│   22                                                                                             │
│   23 # ==== Cargar dataset desde Kedro ====                                                      │
│                                                                                                  │
│ c:\Users\goku8\OneDrive\Escritorio\MCHL -                                                        │
│ copiaaaaa\MLY0100-Parcial-1\.venv\Lib\site-packages\kedro\framework\context\context.py:206 in    │
│ catalog                                                                                          │
│                                                                                                  │
│   203 │   │   │   KedroContextError: Incorrect catalog registered for the project.               │
│   204 │   │                                                                                      │
│   205 │   │   """                                                                                │
│ ❱ 206 │   │   return self._get_catalog()                                                         │
│   207 │                                                                                          │
│   208 │   @property                                                                              │
│   209 │   def params(self) -> dict[str, Any]:                                                    │
│                                                                                                  │
│ c:\Users\goku8\OneDrive\Escritorio\MCHL -                                                        │
│ copiaaaaa\MLY0100-Parcial-1\.venv\Lib\site-packages\kedro\framework\context\context.py:239 in    │
│ _get_catalog                                                                                     │
│                                                                                                  │
│   236 │   │                                                                                      │
│   237 │   │   """                                                                                │
│   238 │   │   # '**/catalog*' reads modular pipeline configs                                     │
│ ❱ 239 │   │   conf_catalog = self.config_loader["catalog"]                                       │
│   240 │   │   # turn relative paths in conf_catalog into absolute paths                          │
│   241 │   │   # before initializing the catalog                                                  │
│   242 │   │   conf_catalog = _convert_paths_to_absolute_posix(                                   │
│                                                                                                  │
│ c:\Users\goku8\OneDrive\Escritorio\MCHL -                                                        │
│ copiaaaaa\MLY0100-Parcial-1\.venv\Lib\site-packages\kedro\config\omegaconf_config.py:224 in      │
│ __getitem__                                                                                      │
│                                                                                                  │
│   221 │   │   else:                                        

# 2. Selección de variables para clustering
Se eliminan columnas que sean categóricas o irrelevantes para K-Means.

In [ ]:
# Quitamos columnas no numéricas
X = df.select_dtypes(include=['float64', 'int64'])
X.head()

# 3. Escalamiento de datos
K-Means requiere que los datos estén escalados.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:5]

# 4. Aplicación de K-Means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

df['cluster'] = clusters
df[['cluster']].value_counts()

# 5. Visualización PCA para reducir a 2D

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
components = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,5))
plt.scatter(components[:,0], components[:,1], c=clusters, cmap='viridis', s=10)
plt.title("Clustering K-Means con PCA (2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# 6. Guardar resultados (opcional)

In [ ]:
df.to_csv("diabetes_clusters_local.csv", index=False)
df.head()